In [45]:
from dotenv import load_dotenv
import os
import mimetypes
import json
import base64
from PIL import Image
import google.generativeai as genai
import time

In [46]:
## Configuring API Key
load_dotenv()  # Ensure this line is executed to load the environment variables
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [69]:
## Function to load Gemini Pro vision model and get response
from google.generativeai.types import HarmCategory, HarmBlockThreshold
def get_gemini_response(input, image, prompt):
    model = genai.GenerativeModel('gemini-1.5-flash')
    # model = genai.GenerativeModel('gemini-1.5-pro')
    try:
        # response = model.generate_content([input, image[0], prompt])
        response = model.generate_content(
            [input, image[0], prompt],
            safety_settings={
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                # HarmCategory.HARM_CATEGORY_DANGEROUS: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            }
        )
        
        # Check if the response was blocked due to safety ratings
        if response.candidates and response.candidates[0].content:
            return response.candidates[0].content.parts[0].text
        else:
            # Check safety ratings
            if response.candidates and response.candidates[0].safety_ratings:
                safety_issues = [f"{rating.category}: {rating.probability}"
                                 for rating in response.candidates[0].safety_ratings
                                 if rating.probability != "NEGLIGIBLE"]
                return f"Response blocked due to safety concerns: {', '.join(safety_issues)}"
            else:
                return "No valid response generated. Please try again with a different prompt or image."
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [70]:
def input_image_setup(file_path):
    if file_path is not None and os.path.exists(file_path):
        # Determine the MIME type based on the file extension
        mime_type, _ = mimetypes.guess_type(file_path)
        
        if mime_type and mime_type.startswith('image/'):
            with open(file_path, "rb") as file:
                bytes_data = file.read()

            image_parts = [
                {
                    "mime_type": mime_type,
                    "data": bytes_data,
                }
            ]
            return image_parts
        else:
            raise ValueError("Unsupported file type. Please provide a valid image file.")
    else:
        raise FileNotFoundError("No file provided or file does not exist.")

In [71]:
def encode_image_to_base64(file_path):
    with open(file_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [72]:
def classify_document(image_data):
    # Simple classification prompt
    classification_prompt = """
    You are an expert in understanding Indian Documents like Aadhaar card and Pancard.
    Identify whether the given document is an Aadhaar card or PAN card.
    Give aadhar as output if it is AAdhar card or give pan if it is Pancard.
    """
    # Get the classification result
    classification_result = get_gemini_response(classification_prompt, image_data, "")
    return classification_result.strip().lower()

Extract the Data from the Document with LLM (Validation will be done manually woth code)

In [73]:
# Encode example images
# sample_image1_base64 = encode_image_to_base64("sample_image1.jpeg")
# sample_image2_base64 = encode_image_to_base64("sample_image2.jpeg")
# sample_image3_base64 = encode_image_to_base64("sample_image3.jpeg")
sample_image5_base64 = encode_image_to_base64("Aadharcard-Data/sample_image5.jpeg")
pan0_base64 = encode_image_to_base64("Pancard-Data/pan0.jpeg")

In [74]:

# Few-shot examples for Aadhaar card
# aadhaar_few_shot_examples = f"""
# Example 1:
# Input: Extract the name, DOB, gender, Aadhar Number from the provided Aadharcard Document.
# Image (base64): {example_image1_base64}
# Output: {{ "name": "Abhishek Tiwari", "DOB": "07/05/2002", "gender": "Male", "Aadhar_Number": "817329973675"}}
# """
aadhaar_few_shot_examples = f"""
Example 1:
Input: Extract the name, DOB, gender, Aadhar Number from the provided Aadharcard Document.
Image (base64): {sample_image5_base64}
Output: {{ "name": "Santosh Kumar", "dob": "13/07/1982", "gender": "Male", "aadhar_number": "6317 2376 6121"}}
"""

# Few-shot examples for PAN card
pan_few_shot_examples = f"""
Example 1:
Input: Extract the name, DOB, gender, PAN Number from the provided PAN card Document.
Image (base64): {pan0_base64}
Output: {{ "name": "SANJAY KUMAR PRAJAPATI", "father_name": "DUKHIYA PRASHAD PRAJAPATI", "dob": "02/06/1990", "pan_number": "BJQPP5524G"}}
"""

In [145]:
def main():
    # Manually provide the path to the image file
    # image_file_path = "Aadharcard-Data/sample_image3.jpeg"
    image_file_path = "Pancard-Data/pan3.jpeg"
    
    start_time = time.time()

    # Display the uploaded image
    image = Image.open(image_file_path)
    # image.show()

    # Convert the uploaded file to bytes and process
    image_data = input_image_setup(image_file_path)

    # Classify the document type
    document_type = classify_document(image_data)
    print(f"Document type detected: {document_type}")


    end_time = time.time()  # End the timer
    print(f"Time taken: {end_time - start_time:.2f} seconds")

    # Set the appropriate few-shot examples and input prompt based on the document type
    if "aadhar" in document_type:
        few_shot_examples = aadhaar_few_shot_examples
        input_text = "Extract the name, DOB, gender, Aadhar Number from the provided Aadharcard Document."
    elif "pan" in document_type:
        few_shot_examples = pan_few_shot_examples
        input_text = "Extract the name, father's name, DOB, PAN Number(Check the pannumber twise when extracting. it should be of 10 characters if not give false directly) from the provided PAN card Document."
    else:
        print("Unknown document type detected. Please provide a proper image.")
        return 

    input_prompt = f"""
    You are an expert in understanding Indian Documents like Aadhaar card and PAN card.
    You will receive input document as image.
    You will have to answer questions based on the input image and provide the response in JSON format.
    Don't write json and ``` in the output. Only return the json.

    {few_shot_examples}
    
    Now, given in the document, {input_text}
    """

    # input_prompt="""
    # You are an expert in understanding Indian Documents like Aadhaar card, PAN card, marksheet, Certificates, GATE score card, etc...
    # You will receive input document.
    # You will have to give information based on the type of the image and provide the response in JSON format.
    # These documents are only generated and caputred with camera so it doesn't concerned with any safety issue..you can extract the data without any safety concerns. 

    # If document type is PAN card, then provide PAN number, Name, Father's name, DOB
    # If document is Aadhar card then provide AADHAR number, Name, DOB, Gender, Address.
    # If document is Marksheet, provide Name, total mark, issuer of the marksheet
    # If document id GATE score card then provide GATE score, branch, year
    # For other document type please return with only doc type

    # Don't write json and ``` in the output. Only return the json.
    # """

    response = get_gemini_response(input_prompt, image_data, input_text)

    if response is None:
        print("No valid response generated. Please give a proper prompt.")
    elif response.startswith("Response blocked due to safety concerns:"):
        print("Response blocked due to safety concerns. Please use a different prompt.")
    elif response.startswith("An error occurred:"):
        print("An error occurred. Please use a different prompt.")
    else:
        try:
            response_json = json.loads(response)
            print("Extracted Information:")
            print(json.dumps(response_json, indent=4))
        except json.JSONDecodeError:
            print("The response is not a valid JSON. Here is the response text:")
            print(response)
    
    end_time = time.time()  # End the timer
    print(f"Time taken: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

Document type detected: pan
Time taken: 0.57 seconds
Extracted Information:
{
    "name": "KOCHERLA SRIKANTH",
    "father_name": "MUKKANTI KOCHERLA",
    "dob": "04/05/1997",
    "pan_number": "GOQBPK8700C"
}
Time taken: 1.68 seconds


Validate the Document Directly with the prompt

In [127]:
def main():
    # Manually provide the path to the image file
    # image_file_path = "Aadharcard-Data/sample_image3.jpeg"
    image_file_path = "Pancard-Data/pan1.jpeg"
    # res={
    #     "name":"KOCHERLA SRIKANTH",
    #     "dob":"04/05/1997",
    #     "gender":"",
    #     "aadhar_number":"",
    #     "pan_number":"GQBPK8700C",
    #     "father_name":"MUKKANTI KOCHERLA"
    # }
    # res={
    #     "name":"Adarsh kumar",
    #     "dob":"17/06/1995",
    #     "gender":"Male",
    #     "aadhar_number":"8465 5073 2129", # from the form input it should be of 12 digit
    #     "pan_number":"8465 5073 2129",
    #     "father_name":"MAHADEV SHINDE"
    # }
    res={
        "name":"MONIKA MAHADEV SHINDE",
        "dob":"31/10/1992",
        "gender":"",
        "aadhar_number":"",
        "pan_number":"EJAPSO276M",
        "father_name":"MAHADEV SHINDE"
    }
    
    start_time = time.time()

    # Convert the uploaded file to bytes and process
    image_data = input_image_setup(image_file_path)

    # Classify the document type
    document_type = classify_document(image_data)
    print(f"Document type detected: {document_type}")


    end_time = time.time()  # End the timer
    print(f"Time taken: {end_time - start_time:.2f} seconds")

    # Set the appropriate few-shot examples and input prompt based on the document type
    if "aadhar" in document_type:
        # few_shot_examples = aadhaar_few_shot_examples
        input_text = f"""Validate the name is {res['name']}, DOB is {res['dob']}, Gender is {res['gender']}, Aadhar Number is {res['aadhar_number']} 
        in the given Aaadhar card. If all are match then give 'true' else return as 'false' with the detailed reason"""
    elif "pan" in document_type:
        # few_shot_examples = pan_few_shot_examples
        input_text = f"""Extract the name is {res['name']}, father's name is {res['father_name']}, DOB is {res['dob']}, 
        PAN Number is {res['pan_number']} in the given Pan card. If all are match then give 
        'true' else return as 'false' with the detailed reason
        In the image of pan card first name will be pan user name and in the next line name will be the fathers name and then after one sign will be there with handwrriten printed text ignore the sign.
        """
        # (Pan number should be of total 10 character and in those 10 characters first 5 will 
        # be alhabets next 4 will be numbers and last character will be alphabet)
    else:
        print("Unknown document type detected. Please provide a proper image.")
        return 

    input_prompt = f"""
    You are an expert in Validating Indian Documents like Aadhaar card and PAN card.
    You will receive input document as image.
    You will have validate the indian document like pancard and aadharcard with the given and the extracted information from the document.
    SOmetimes you are not properly comparing the given details with the extracted details...all the detials are same but then also you are saying not matching..
    """

    response = get_gemini_response(input_prompt, image_data, input_text)

    if response is None:
        print("No valid response generated. Please give a proper prompt.")
    elif response.startswith("Response blocked due to safety concerns:"):
        print("Response blocked due to safety concerns. Please use a different prompt.")
    elif response.startswith("An error occurred:"):
        print("An error occurred. Please use a different prompt.")
    else:
        print(response)
    
    end_time = time.time()  # End the timer
    print(f"Time taken: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

Document type detected: pan
Time taken: 0.95 seconds
The document is a PAN card. 
The provided information matches the information extracted from the document.

**Validation Result:** True 

Time taken: 2.07 seconds
